## Import Libraries

In [1]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn
import random
from sklearn.preprocessing import StandardScaler


random.seed(42)

## Read Data

In [2]:
axial_image = 'MRNet-v1.0/train/axial/0000.npy'
coronal_image = 'MRNet-v1.0/train/coronal/0000.npy'
saggital_image = 'MRNet-v1.0/train/sagittal/0000.npy'


img_array_axial = np.load(axial_image)
img_array_coronal = np.load(coronal_image)
img_array_sagittal = np.load(saggital_image)

In [3]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'


In [4]:
def load_labels(abnormal_path, acl_path, meniscus_path):
    
    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])
    
    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))
    
    return df_lables_combined

In [5]:
# Load Train Labels
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,0000,1,0,0
1,0001,1,1,1
2,0002,1,0,0
3,0003,1,0,1
4,0004,1,0,0
...,...,...,...,...
1125,1125,1,0,1
1126,1126,1,0,1
1127,1127,0,0,0
1128,1128,1,0,0


In [6]:
# Load Valid Labels
df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [7]:
glob.glob('MRNet-v1.0\*')

['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [8]:
# Read Traning Data
df_labels_train.pop("Image_Name")
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []
    
    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')
    
    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)
        
        for i in train_data:
#             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
            if 'coronal' in i:
                coronal_images.append(image_tensor)
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images

train_path =  'MRNet-v1.0\\train'
valid_path =  'MRNet-v1.0\\valid'

axial_images_train, coronal_images_train, sagittal_images_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\axial
Loading Complete From :    MRNet-v1.0\train\axial



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\coronal
Loading Complete From :    MRNet-v1.0\train\coronal



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\sagittal
Loading Complete From :    MRNet-v1.0\train\sagittal





In [9]:
yTrain = df_labels_train.pop('Abnormal_Label')
yTest = df_labels_valid.pop('Abnormal_Label')

In [10]:
print(np.array(axial_images_train).shape)
print(np.array(coronal_images_train).shape)
print(np.array(sagittal_images_train).shape)

(1130, 256, 256, 16)
(1130, 256, 256, 16)
(1130, 256, 256, 16)


## Modeling

In [12]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [24]:
# Three Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=15, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)

    flat1 = Flatten()(pool12)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()
model_3, Visible_3 = model()

merge = concatenate([model_1, model_2])

hidden1 = Dense(100, activation='relu')(merge)
hidden2 = Dense(50, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2, Visible_3], outputs=output)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_7 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 252, 252, 15  6015        ['input_6[0][0]']                
                                )                                                           

In [26]:
lr = 0.00005
momentum = 0.6
epochs = 20
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = lr, momentum = momentum),metrics=["binary_accuracy"])

model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train), np.array(axial_images_train)],
          y = yTrain,
          epochs=epochs,
          batch_size=32,
          validation_split=0.20)


Epoch 1/20
29/29 [==============================] - 61s 2s/step - loss: 0.0237 - binary_accuracy: 1.0000 - val_loss: 0.6534 - val_binary_accuracy: 0.8274
Epoch 2/20
29/29 [==============================] - 54s 2s/step - loss: 0.0218 - binary_accuracy: 1.0000 - val_loss: 0.6322 - val_binary_accuracy: 0.8097
Epoch 3/20
29/29 [==============================] - 53s 2s/step - loss: 0.0202 - binary_accuracy: 1.0000 - val_loss: 0.6735 - val_binary_accuracy: 0.8319
Epoch 4/20
29/29 [==============================] - 53s 2s/step - loss: 0.0190 - binary_accuracy: 1.0000 - val_loss: 0.6525 - val_binary_accuracy: 0.8186
Epoch 5/20
29/29 [==============================] - 55s 2s/step - loss: 0.0175 - binary_accuracy: 1.0000 - val_loss: 0.6587 - val_binary_accuracy: 0.8319
Epoch 6/20
29/29 [==============================] - 53s 2s/step - loss: 0.0164 - binary_accuracy: 1.0000 - val_loss: 0.6685 - val_binary_accuracy: 0.8230
Epoch 7/20
29/29 [==============================] - 53s 2s/step - loss: 0.01

In [27]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11011857856274166967
xla_global_id: -1
]


In [28]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [29]:
axial_images_valid, coronal_images_valid, sagittal_images_valid = read_data(valid_path, df_labels_valid.astype('float32'))

Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\axial
Loading Complete From :    MRNet-v1.0\valid\axial



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\coronal
Loading Complete From :    MRNet-v1.0\valid\coronal



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\sagittal
Loading Complete From :    MRNet-v1.0\valid\sagittal





## Evaluate Model

In [31]:
y_pred = model.predict([ np.array(coronal_images_valid), np.array(sagittal_images_valid), np.array(axial_images_valid)])

4/4 [==============================] - 1s 273ms/step


In [32]:
y_pred

array([[0.49918014],
       [0.9250235 ],
       [0.49767265],
       [0.15310729],
       [0.8911452 ],
       [0.9997275 ],
       [0.9119148 ],
       [0.99374586],
       [0.82053685],
       [0.790589  ],
       [0.9996654 ],
       [0.99996173],
       [0.9591416 ],
       [0.99999493],
       [0.9999955 ],
       [0.93521047],
       [0.19040617],
       [0.21184956],
       [0.11607963],
       [0.96539277],
       [0.00330325],
       [0.9996408 ],
       [0.6646743 ],
       [0.93198276],
       [0.99534035],
       [0.9965625 ],
       [0.99559104],
       [0.1512655 ],
       [0.99999535],
       [0.4579554 ],
       [0.9863014 ],
       [0.999305  ],
       [0.9933658 ],
       [0.99999166],
       [0.42238578],
       [0.28066435],
       [0.99863803],
       [0.5772969 ],
       [0.62623787],
       [0.9988505 ],
       [0.9999759 ],
       [0.9999968 ],
       [0.90036285],
       [0.9906236 ],
       [0.9991785 ],
       [0.9798644 ],
       [0.9995564 ],
       [0.999

In [34]:
from sklearn.metrics import roc_auc_score, accuracy_score
ypreds_abnormal_soft = list(y_pred[:,0])
ypreds_abnormal_hard = list(np.array(y_pred[:,0]).round())
ytrue_abnormal = yTest #(np.array(df_labels_valid)[:,0])
auc_abnormal = roc_auc_score(y_score= ypreds_abnormal_soft, y_true= ytrue_abnormal)
accuracy_abnormal = accuracy_score(y_true= ytrue_abnormal, y_pred= ypreds_abnormal_hard)

logString = "LR: " + str(lr) + "\nMomentum: " + str(momentum) + "\nEpochs: " + str(epochs) + "\nAbnormal MRI AUC: " + str(auc_abnormal) + "\nAbnormal MRI Accuracy: " + str(accuracy_abnormal)+"\n\n" #"\nACL Injury AUC: " + str(auc_acl) + "\nMeniscus Injury AUC: " + str(auc_meniscus) +"\nAbnormal MRI Accuracy: " + str(accuracy_abnormal) + "\nACL Injury Accuracy: " + str(accuracy_acl) + "\nMeniscus Injury Accuracy: " + str(accuracy_meniscus) +"\n\n"

print (logString)

filename = "Logs.txt"
file = open(filename, "a")

file.write(logString)
file.close()



LR: 5e-05
Momentum: 0.6
Epochs: 20
Abnormal MRI AUC: 0.7313684210526316
Abnormal MRI Accuracy: 0.7916666666666666




In [35]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

import torch
print(torch.cuda.is_available())

False
False
False
